<a href="https://colab.research.google.com/github/Anna100ianova/A-B-testing/blob/main/A_B_testing_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Завантаження даних


In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import pandas as pd
import numpy as np
import matplotlib as plt


project_id = "data-analytics-mate"
dataset_id = "DA"

client = bigquery.Client(project=project_id)

query = """
with session_info as(
  SELECT
  s.date,
  s.ga_session_id,
  sp.country,
  sp.device,
  sp.continent,
  sp.channel,
  ab.test,
  ab.test_group
  FROM `data-analytics-mate.DA.ab_test` as ab
  JOIN `data-analytics-mate.DA.session` as s
  ON ab.ga_session_id = s.ga_session_id
  JOIN `data-analytics-mate.DA.session_params` as sp
  ON ab.ga_session_id = sp.ga_session_id
),
session_with_orders as(


SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  count(distinct o.ga_session_id) as session_with_orders
FROM `data-analytics-mate.DA.order` as o
JOIN session_info
ON o.ga_session_id = session_info.ga_session_id
GROUP BY
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group
  ),
events as(
  SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  ep.event_name,
  count(ep.ga_session_id) as event_cnt
  FROM `data-analytics-mate.DA.event_params` as ep
  join session_info
  on session_info.ga_session_id = ep.ga_session_id
  GROUP BY
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  ep.event_name
  ),
  session as(
  Select
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  count(distinct session_info.ga_session_id) as session_cnt
  From session_info
  group by
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group
  ),
account as(
  SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  count(distinct acs.ga_session_id) as new_account_cnt
  FROM `data-analytics-mate.DA.account_session` as acs
  join session_info
  on acs.ga_session_id = session_info.ga_session_id
  group by
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group
  )
SELECT
  session_with_orders.date,
  session_with_orders.country,
  session_with_orders.device,
  session_with_orders.continent,
  session_with_orders.channel,
  session_with_orders.test,
  session_with_orders.test_group,
  "session with orders" as event_name,
  session_with_orders.session_with_orders as value
From session_with_orders
union all
SELECT
  events.date,
  events.country,
  events.device,
  events.continent,
  events.channel,
  events.test,
  events.test_group,
  event_name,
  event_cnt as value
From events
union all
SELECT
  session.date,
  session.country,
  session.device,
  session.continent,
  session.channel,
  session.test,
  session.test_group,
  "session" as event_name,
  session_cnt as value
From session
union all
SELECT
  account.date,
  account.country,
  account.device,
  account.continent,
  account.channel,
  account.test,
  account.test_group,
  "new account" as event_name,
  new_account_cnt as value
From account

"""

df = client.query(query).to_dataframe()
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-12-08,Palestine,desktop,Asia,Direct,4,2,new account,1
1,2020-12-08,Palestine,desktop,Asia,Direct,3,2,new account,1
2,2020-11-06,Puerto Rico,desktop,Americas,Social Search,2,2,new account,1
3,2020-11-06,Puerto Rico,desktop,Americas,Social Search,1,1,new account,1
4,2020-12-08,Croatia,desktop,Europe,Direct,4,2,new account,1


##Створення датафрейму з метриками


In [ ]:
grouped_df = df.groupby(["test", "test_group", "event_name"])["value"].sum().reset_index()
grouped_df.head()

,test,test_group,event_name,value
0,1,1,add_payment_info,1988
1,1,1,add_shipping_info,3034
2,1,1,add_to_cart,1395
3,1,1,begin_checkout,3784
4,1,1,click,368


In [ ]:
from statsmodels.stats.proportion import proportions_ztest

# групуємо дані лише за івентами, які нас цікавлять
filtered_events = ["session", "add_payment_info", "add_shipping_info", "begin_checkout", "new account"]
filtered_df_for_pivot = grouped_df[grouped_df["event_name"].isin(filtered_events)]

# робимо з даних зведену таблицю
pivoted_df = filtered_df_for_pivot.pivot_table(
    index=["test", "test_group"],
    columns="event_name",
    values="value",
    fill_value=0
).reset_index()

# переназиваємо стовбці, щоб за логікою було зрозуміло, що це загальна кількість сесій
pivoted_df = pivoted_df.rename(columns={
    "session": "session_cnt",
    "add_payment_info": "add_payment_info_cnt",
    "add_shipping_info": "add_shipping_info_cnt",
    "begin_checkout": "begin_checkout_cnt",
    "new account": "new_account_cnt"
})

pivoted_df.head()

event_name,test,test_group,add_payment_info_cnt,add_shipping_info_cnt,begin_checkout_cnt,new_account_cnt,session_cnt
0,1,1,1988.0,3034.0,3784.0,3823.0,45362.0
1,1,2,2229.0,3221.0,4021.0,3681.0,45193.0
2,2,1,2344.0,3480.0,4262.0,4165.0,50637.0
3,2,2,2409.0,3510.0,4313.0,4184.0,50244.0
4,3,1,3623.0,5298.0,9532.0,5856.0,70047.0


##Підготовка даних та проведення z-test

In [ ]:
from statsmodels.stats.proportion import proportions_ztest
from google.colab import files

alpha = 0.05

metrics_to_analyze = [
    "add_payment_info_cnt",
    "add_shipping_info_cnt",
    "begin_checkout_cnt",
    "new_account_cnt"
]

final_results = []

# групуємо по test
for test_val, test_data in pivoted_df.groupby("test"):

    # перевіряємо чи є обидві групи
    if test_data["test_group"].nunique() != 2:
        continue

    group_1 = test_data[test_data["test_group"] == 1].iloc[0]
    group_2 = test_data[test_data["test_group"] == 2].iloc[0]

    session_cnt_g1 = group_1["session_cnt"]
    session_cnt_g2 = group_2["session_cnt"]

    if session_cnt_g1 == 0 or session_cnt_g2 == 0:
        continue

    for metric_col in metrics_to_analyze:

        metric_name = metric_col.replace("_cnt", "") # заміняємо назви на більш логічні, бо буде загальна кількість

        count_g1 = group_1[metric_col]
        count_g2 = group_2[metric_col]

        # конверсії
        cr_g1 = count_g1 / session_cnt_g1
        cr_g2 = count_g2 / session_cnt_g2

        # z-test
        z_stat, p_value = proportions_ztest(
            count=[count_g1, count_g2],
            nobs=[session_cnt_g1, session_cnt_g2],
            alternative="two-sided"
        )


        metric_change = (
            (cr_g2 - cr_g1) / cr_g1 * 100
            if cr_g1 != 0 else np.nan
        )

        significant = (
            "Так" if p_value < alpha
            else "Ні"
        )

        final_results.append({
            "test_number": test_val,
            "metric": metric_name,
            "numenator_event": metric_col,
            "denominator_event": "session_cnt",
            "numenator_conversion_1": count_g1,
            "denominator_conversion_1": session_cnt_g1,
            "conversion_rate_1": cr_g1,
            "numenator_conversion_2": count_g2,
            "denominator_conversion_2": session_cnt_g2,
            "conversion_rate_2": cr_g2,
            "metric_change_%": metric_change,
            "z_stat": z_stat,
            "p_value": p_value,
            "significant": significant
        })

final_df = pd.DataFrame(final_results)

final_df.head(30)


,test_number,metric,numenator_event,denominator_event,numenator_conversion_1,denominator_conversion_1,conversion_rate_1,numenator_conversion_2,denominator_conversion_2,conversion_rate_2,metric_change_%,z_stat,p_value,significant
0,1,add_payment_info,add_payment_info_cnt,session_cnt,1988.0,45362.0,0.043825,2229.0,45193.0,0.049322,12.542021,-3.924884,0.000087,Так
1,1,add_shipping_info,add_shipping_info_cnt,session_cnt,3034.0,45362.0,0.066884,3221.0,45193.0,0.071272,6.560481,-2.603571,0.009226,Так
2,1,begin_checkout,begin_checkout_cnt,session_cnt,3784.0,45362.0,0.083418,4021.0,45193.0,0.088974,6.660587,-2.978783,0.002894,Так
3,1,new_account,new_account_cnt,session_cnt,3823.0,45362.0,0.084278,3681.0,45193.0,0.081451,-3.354299,1.542883,0.122859,Ні
4,2,add_payment_info,add_payment_info_cnt,session_cnt,2344.0,50637.0,0.046290,2409.0,50244.0,0.047946,3.576911,-1.240994,0.214608,Ні
5,2,add_shipping_info,add_shipping_info_cnt,session_cnt,3480.0,50637.0,0.068724,3510.0,50244.0,0.069859,1.650995,-0.709557,0.477979,Ні
6,2,begin_checkout,begin_checkout_cnt,session_cnt,4262.0,50637.0,0.084168,4313.0,50244.0,0.085841,1.988164,-0.952898,0.340642,Ні
7,2,new_account,new_account_cnt,session_cnt,4165.0,50637.0,0.082252,4184.0,50244.0,0.083274,1.241934,-0.588793,0.556000,Ні
8,3,add_payment_info,add_payment_info_cnt,session_cnt,3623.0,70047.0,0.051722,3697.0,70439.0,0.052485,1.474630,-0.643172,0.520112,Ні
9,3,add_shipping_info,add_shipping_info_cnt,session_cnt,5298.0,70047.0,0.075635,5188.0,70439.0,0.073652,-2.621211,1.413727,0.157442,Ні


##Висновки по a/b test в розрізі:
* **додавання платіжної інформації**
* **додавання інформації для доставки**
* **оформлення замовлення**
* **новий акаунт**

1. Перший тест - статистично значуще зростання по шляху оплата -> покупка.Це найрезультативніший експеримент. Усі ключові метрики показали статистично значуще зростання (p-value < 0.05).
* **Add Payment Info**: Конверсія зросла на 12.5%.
* **Add Shipping Info**: Зростання на 6.6%.
* **Begin Checkout**: Зростання на 6.4%.
* **Кінцева конверсія** в покупку зросла на 8.3%.
2. Другий тест - тут ми бачимо нейтральну ситуацію. Хоча цифри дещо коливаються (наприклад, +2.3% по покупках), високий p-value (0.33) каже нам, що ці зміни випадкові. Варіант Б не кращий за А.
3. Третій тест - показав негативну динаміку за ключовими етапами воронки:
* **Add Shipping Info**: Падіння конверсії на -2.6% p-value = 0.15. Хоча це не є на 100% статистично підтвердженим провалом, тренд явно негативний.
* **Конверсія в покупку** впала на -1.3%.
4. Тест четвертий - можемо назвати провальним, бачимо статистично значуще погіршення деяких метрик:
* **Add Payment Info**: -6.4% (p-value = 0.0003) — критичне падіння.
* **Begin Checkout**: -6.2% (p-value = 0.000003) — вкрай висока впевненість у тому, що зміна шкодить.
* **Конверсія в покупку** впала на -3.2%.


---
Робимо висновки, що гіпотезу першого тесту впроваджуємо, що забезпечить нам приріст конверсії на 8.3%. Тест №4 зупиняємо та аналізуємо, що могло призвести до таких наслідків. Тести №3 та №2 архівуємо або доробляємо.


##Аналіз статистичної значущості в розрізі івентів: add_to_cart, select_item, session with orders та девайсів і каналів

In [ ]:
segmentation_keys = ["test", "test_group", "device", "channel"] # сегменти, в розрізі яких я хочу робити аналіз

desired_event_names = [
    "session with orders",
    "add_to_cart",
    "select_item",
    "session"
]  # назва івентів, в розрізі яких будемо дивитись статистичну значущість змін між А та В

segmented_grouped_df = df.groupby(segmentation_keys + ["event_name"])["value"].sum().reset_index()

# фільтрую "event_name", щоб залишились лише ті, що мене цікавлять
filtered_segmented_grouped_df = segmented_grouped_df[
    segmented_grouped_df["event_name"].isin(desired_event_names)
]

pivoted_segmented_df = filtered_segmented_grouped_df.pivot_table(
    index=segmentation_keys,
    columns="event_name",
    values="value",
    fill_value=0
).reset_index()
# переназиваємо стовбці, щоб за логікою було зрозуміло, що це загальна кількість сесій
pivoted_segmented_df = pivoted_segmented_df.rename(columns={
    "session": "session_cnt",
    "add_to_cart": "add_to_cart_cnt",
    "select_item": "select_item_cnt",
    "session with orders": "session_with_orders_cnt"
})

pivoted_segmented_df.head()

event_name,test,test_group,device,channel,add_to_cart_cnt,select_item_cnt,session_cnt,session_with_orders_cnt
0,1,1,desktop,Direct,179.0,63.0,6258.0,641.0
1,1,1,desktop,Organic Search,251.0,130.0,9221.0,906.0
2,1,1,desktop,Paid Search,241.0,70.0,6843.0,706.0
3,1,1,desktop,Social Search,62.0,24.0,2232.0,211.0
4,1,1,desktop,Undefined,88.0,28.0,1913.0,214.0


In [ ]:

from scipy.stats import norm

metrics = {
    "add_to_cart_rate": "add_to_cart_cnt",
    "select_item_rate": "select_item_cnt",
    "order_rate": "session_with_orders_cnt"
}

results = []

group_cols = ["test", "device", "channel"]

for (test_val, device_val, channel_val), group in pivoted_segmented_df.groupby(group_cols):

    # перевірка що є 2 групи
    if group["test_group"].nunique() != 2:
        continue

    group_A = group[group["test_group"] == 1].iloc[0]
    group_B = group[group["test_group"] == 2].iloc[0]

    for metric_name, numerator_col in metrics.items():

        num_1 = group_A[numerator_col]
        num_2 = group_B[numerator_col]

        den_1 = group_A["session_cnt"]
        den_2 = group_B["session_cnt"]

        cr_1 = num_1 / den_1
        cr_2 = num_2 / den_2

        # pooled proportion
        p_pool = (num_1 + num_2) / (den_1 + den_2)

        se = np.sqrt(p_pool * (1 - p_pool) * (1/den_1 + 1/den_2))

        if se == 0:
            continue

        z_stat = (cr_1 - cr_2) / se
        p_value = 2 * (1 - norm.cdf(abs(z_stat)))

        significant = "Так" if p_value < 0.05 else "Ні"

        metric_change = ((cr_2 - cr_1) / cr_1) if cr_1 != 0 else np.nan

        results.append([
            test_val,
            device_val,
            channel_val,
            metric_name,
            numerator_col,
            "session_cnt",
            num_1,
            den_1,
            cr_1,
            num_2,
            den_2,
            cr_2,
            metric_change,
            z_stat,
            p_value,
            significant
        ])

final_df = pd.DataFrame(results, columns=[
    "test",
    "device",
    "channel",
    "metric",
    "numerator_event",
    "denominator_event",
    "numerator_conversion_1",
    "denominator_conversion_1",
    "conversion_rate_1",
    "numerator_conversion_2",
    "denominator_conversion_2",
    "conversion_rate_2",
    "metric_change",
    "z_stat",
    "p_value",
    "significant"
])

final_df.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Висновки по A/B test в розрізі:
* **сессій з замовленнями**
* **додавання до кошику**
* **вибір товару**

Також в розрізі тестів, девайсів та каналів

1. Перший тест - середня зміна: -1.1% Майже 27% метрик статистично значущі.Ефект нестабільний, частіше нейтральний. Тест не дав системного покращення. Є окремі сегменти з ефектом, але глобального покращення немає.
2. Другий тест - середня зміна: +58%. 20% значущих результатів. Дуже сильне покращення у частині сегментів. Є потужний ефект, але не всюди стабільний. Ймовірно тест працює тільки в окремих каналах/девайсах.
3. Третій тест - середня зміна: +5.8%. 31% значущих результатів.Найбільш стабільний позитивний тест. Найбільш збалансоване покращення.
4. Четвертий тест - середня зміна: -8.3%. 47% значущих результатів (багато негативних). Тест погіршує показники. Не запускати у прод.

###Аналіз по девайсах
* **Desktop**	- середня зміна +1.9%. Частка значущих	23%.
* **Mobile** - середня зміна +6.1%. Частка значущих 32%.
* **Tablet** - середня зміна +32.7%. Частка значущих	38%.
Mobile реагує краще, ніж desktop. Tablet показує найбільший приріст (але можливо мала вибірка). Desktop - найслабший ефект.

###Аналіз по каналах
* **Direct** - середня зміна -5.5%. Частка значущих	22%
* **Organic Search** - середня зміна	-2.7%. Частка значущих	39%
* **Paid Search** - середня зміна	+10.3%. Частка значущих	22%
* **Social Search** - середня зміна	+30%. Частка значущих	44%
* **Undefined** - середня зміна	+35%.Частка значущих	28%
Social Search - найсильніший позитивний ефект. Paid Search - стабільний плюс. Direct та Organic - слабкий або негативний ефект

[Посилання на csv таблицю із результатами](https://drive.google.com/file/d/1Ikbm2s8X_caiciELKne20blbVm6xiQ_e/view?usp=sharing)

[Посилання на візуалізацію в табло](https://public.tableau.com/views/A_BTest_17709264130560/ABTest?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)

